<a href="https://colab.research.google.com/github/NikolayWTF/Hackathon-Perm/blob/main/%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_%D0%9F%D0%B5%D1%80%D0%BC%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1AIIIKnNWGWTxIRdEw1OzWNeIs7h6QOdV
!gdown 1X9ZySLYXzoidGfGc5XThHu8X1xaz4t7P
!gdown 1uvsxiiGY8xHatb0HnxxnHznMgQDD2Txc
!gdown 1IG8jDibA-zy5v-8s_yFnZBnEHc6_sxTD
!gdown 1SLSp9FD8VBOkLY0gGoAk0TFsLc7StvYD

Downloading...
From: https://drive.google.com/uc?id=1AIIIKnNWGWTxIRdEw1OzWNeIs7h6QOdV
To: /content/yolov8s_180epochs.pt
100% 22.5M/22.5M [00:00<00:00, 148MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1X9ZySLYXzoidGfGc5XThHu8X1xaz4t7P
To: /content/yolov8n_150epochs.pt
100% 6.22M/6.22M [00:00<00:00, 17.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uvsxiiGY8xHatb0HnxxnHznMgQDD2Txc
To: /content/yolov8n_20epochs.pt
100% 6.22M/6.22M [00:00<00:00, 194MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IG8jDibA-zy5v-8s_yFnZBnEHc6_sxTD
To: /content/s_5.pt
100% 22.5M/22.5M [00:00<00:00, 44.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SLSp9FD8VBOkLY0gGoAk0TFsLc7StvYD
To: /content/video0.rar
100% 86.9M/86.9M [00:00<00:00, 106MB/s]


In [5]:
!gdown 1JSFVsos9kUgOYesKEK-YM-qnzUIWSLig

Downloading...
From: https://drive.google.com/uc?id=1JSFVsos9kUgOYesKEK-YM-qnzUIWSLig
To: /content/yolov8m_2000epochs.pt
100% 52.0M/52.0M [00:00<00:00, 62.7MB/s]


In [ ]:
!pip install unrar
!pip install ultralytics

In [ ]:
#Распаковываю архив с моделями и добавляю в папку networks
!mkdir "video0"
!unrar x -Y "video0.rar" "video0"

In [4]:
import ultralytics
import os
import math

In [6]:
model = ultralytics.YOLO("yolov8m_2000epochs.pt")
path = "video0/frames_rgb_0/"
RMSD_w, RMSD_p, RMSD_g, RMSD_m = 0, 0, 0, 0
g_w, g_p, g_m, g_g = 0, 0, 0, 0
l_w, l_p, l_m, l_g = 0, 0, 0, 0
T = len(os.listdir(path))
for file in os.listdir(path):
  file_path = path + file
  result = model(file_path)

  # получим объект файла
  file_ans = open("video0/frames_output/" + file[:-3] + "txt", "r")
  lines = file_ans.readlines()
  wood = int(lines[0])
  glass = int(lines[1])
  plastic = int(lines[2])
  metal = int(lines[3])
  g_w += wood
  g_g += glass
  g_p += plastic
  g_m += metal
  file_ans.close

  local_wood = 0
  local_glass = 0
  local_plastic = 0
  local_metal = 0

  for r in result:
          boxes = r.boxes
          if len(boxes) == 0:
              print("Ничего не распознало")
          else:
            for box in boxes:
              x_0 = int(box.xyxy[0][0])
              y_0 = int(box.xyxy[0][1])
              x_1 = int(box.xyxy[0][2])
              y_1 = int(box.xyxy[0][3])
              name = model.names[int(box.cls)]
              if (name == "wood"):
                local_wood += 1
              elif name == "plastic":
                local_plastic += 1
              elif name == "metal":
                local_metal += 1
              elif name == "glass":
                local_glass += 1
              # img = r.orig_img[y_0:y_1, x_0:x_1]
              # cv2.imwrite("tmp.jpg", img)
  RMSD_w += (wood - local_wood)**2
  RMSD_g += (glass - local_glass)**2
  RMSD_p += (plastic - local_plastic)**2
  RMSD_m += (metal - local_metal)**2

  l_w += local_wood
  l_g += local_glass
  l_p += local_plastic
  l_m += local_metal

RMSD_w = math.sqrt(RMSD_w/T)
RMSD_m = math.sqrt(RMSD_m/T)
RMSD_g = math.sqrt(RMSD_g/T)
RMSD_p = math.sqrt(RMSD_p/T)
RMSD =(RMSD_w + RMSD_p + RMSD_g + RMSD_m)/4
print(RMSD)
print("Истиное количество дерева, стекла, пластика, металла: ", g_w, g_g, g_p, g_m)
print("Расчитанное количество дерева, стекла, пластика, металла: ", l_w, l_g, l_p, l_m)


image 1/1 /content/video0/frames_rgb_0/0100.png: 384x640 4 woods, 7 glasss, 3 plastics, 2 metals, 1151.0ms
Speed: 7.2ms preprocess, 1151.0ms inference, 34.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0067.png: 384x640 3 woods, 3 glasss, 4 plastics, 1 metal, 1625.4ms
Speed: 3.2ms preprocess, 1625.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0111.png: 384x640 4 woods, 4 glasss, 3 plastics, 4 metals, 1438.3ms
Speed: 2.4ms preprocess, 1438.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0051.png: 384x640 4 woods, 4 glasss, 2 plastics, 2 metals, 962.3ms
Speed: 2.1ms preprocess, 962.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0093.png: 384x640 4 woods, 4 glasss, 4 plastics, 2 metals, 1000.1ms
Speed: 2.5ms preprocess, 1000.1ms inference, 2.0ms postprocess per

0.949910625570183
Истиное количество дерева, стекла, пластика, металла:  1644 1475 1444 1374
Расчитанное количество дерева, стекла, пластика, металла:  1590 1399 1366 1109


In [22]:
model = ultralytics.YOLO("yolov8n_150epochs.pt")
path = "video0/frames_rgb_0/"

g_w, g_g, g_p, g_m = 0, 0, 0, 0
prev_w, prev_g, prev_p, prev_m = 0, 0, 0, 0
line = 300

T = len(os.listdir(path))
for file_ind, file in enumerate(os.listdir(path)):
  file_path = path + file
  result = model(file_path)

  for r in result:
          local_wood = 0
          local_glass = 0
          local_plastic = 0
          local_metal = 0
          boxes = r.boxes
          if len(boxes) == 0:
              print("Ничего не распознало")
          else:
            for box in boxes:
              x_0 = int(box.xyxy[0][0])
              y_0 = int(box.xyxy[0][1])
              x_1 = int(box.xyxy[0][2])
              y_1 = int(box.xyxy[0][3])
              name = model.names[int(box.cls)]
              if file_ind == 0:

                if x_0 > line:
                  if (name == "wood"):
                    g_w += 1
                  elif name == "plastic":
                    g_p += 1
                  elif name == "metal":
                    g_m += 1
                  elif name == "glass":
                    g_g += 1
                elif (x_0 < line) and (x_1 > line):
                  if (name == "wood"):
                    prev_w += 1
                  elif name == "plastic":
                    prev_p += 1
                  elif name == "metal":
                    prev_m += 1
                  elif name == "glass":
                    prev_g += 1
              elif file_ind == T-1:
                   if (x_1 < line):
                    if (name == "wood"):
                      g_w += 1
                    elif name == "plastic":
                      g_p += 1
                    elif name == "metal":
                      g_m += 1
                    elif name == "glass":
                      g_g += 1
              else:
                if (x_0 < line) and (x_1 > line):
                  if (name == "wood"):
                    local_wood += 1
                  elif name == "plastic":
                    local_plastic += 1
                  elif name == "metal":
                    local_metal += 1
                  elif name == "glass":
                    local_glass += 1
                  if local_wood < prev_w:
                    g_w += prev_w - local_wood
                  if local_plastic < prev_p:
                    g_p += prev_p - local_plastic
                  if local_metal < prev_m:
                    g_m += prev_m - local_metal
                  if local_glass < prev_g:
                    g_g += prev_g - local_glass
                  prev_w, prev_p, prev_m, prev_g = local_wood, local_plastic, local_metal, local_glass

          print(g_w, g_g, g_p, g_m)


image 1/1 /content/video0/frames_rgb_0/0100.png: 384x640 4 woods, 7 glasss, 4 plastics, 1 metal, 199.5ms
Speed: 4.2ms preprocess, 199.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0067.png: 384x640 3 woods, 3 glasss, 5 plastics, 1 metal, 130.2ms
Speed: 1.5ms preprocess, 130.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)



3 3 2 0
3 3 3 0


image 1/1 /content/video0/frames_rgb_0/0111.png: 384x640 4 woods, 5 glasss, 3 plastics, 3 metals, 142.1ms
Speed: 1.7ms preprocess, 142.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0051.png: 384x640 4 woods, 2 glasss, 2 plastics, 2 metals, 134.8ms
Speed: 1.6ms preprocess, 134.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



3 3 4 0
3 3 4 0


image 1/1 /content/video0/frames_rgb_0/0093.png: 384x640 4 woods, 4 glasss, 4 plastics, 2 metals, 132.3ms
Speed: 1.6ms preprocess, 132.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0076.png: 384x640 3 woods, 4 glasss, 7 plastics, 1 metal, 132.4ms
Speed: 1.7ms preprocess, 132.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



...

139 112 207 60
141 112 207 61


image 1/1 /content/video0/frames_rgb_0/0311.png: 384x640 3 woods, 6 glasss, 4 plastics, 5 metals, 132.7ms
Speed: 2.4ms preprocess, 132.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/video0/frames_rgb_0/0082.png: 384x640 6 woods, 3 glasss, 5 plastics, 2 metals, 136.9ms
Speed: 1.6ms preprocess, 136.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



141 112 209 62
141 113 209 62


image 1/1 /content/video0/frames_rgb_0/0166.png: 384x640 3 woods, 5 glasss, 7 plastics, 3 metals, 138.1ms
Speed: 2.8ms preprocess, 138.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


142 117 215 63


In [25]:
wood = (139 + 124 + 121 + 127 + 122 + 134)/6
glass = (119 + 131 + 128 + 126 + 126 + 125)/6
plastic = (182 + 186 + 174 + 195 + 185 + 191)/6
metal = (62 + 67 + 60 + 59 + 67 + 51)/6

# wood, glass, plastic, metal = g_w, g_g, g_p, g_m

# k_wood = wood/67
# k_glass = glass/59
# k_plastic = plastic/57
# k_metal = metal/61

k_wood, k_glass, k_plastic, k_metal = 1.908, 2.133, 3.254, 1

RMSD_w = math.sqrt((67 - wood/k_wood)**2/T)
RMSD_g = math.sqrt((59 - glass/k_glass)**2/T)
RMSD_p = math.sqrt((57 - plastic/k_plastic)**2/T)
RMSD_m = math.sqrt((61 - metal/k_metal)**2/T)
print(RMSD_w, RMSD_g, RMSD_p, RMSD_m)
RMSD = (RMSD_w + RMSD_g + RMSD_p + RMSD_m)/4
print(RMSD)
print(k_wood, k_glass, k_plastic, k_metal)
print("Дерево -",wood)
      , glass, plastic, metal)

7.636035844951529e-05 0.00035006552512954894 0.00036938795401299484 0.0
0.00019895345939801478
1.908 2.133 3.254 1
127.83333333333333 125.83333333333333 185.5 61.0


In [24]:
!yolo predict model=yolov8n_150epochs.pt source='video0/frames_rgb_0/0000.png'

Ultralytics YOLOv8.0.202 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/video0/frames_rgb_0/0000.png: 384x640 3 woods, 1 glass, 9 plastics, 2 metals, 218.3ms
Speed: 7.0ms preprocess, 218.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
